In [1]:
!pip install duckdb

# All in one function

In [7]:
import pandas as pd
import json
import duckdb
import altair as alt

con = duckdb.connect("flattened.duckdb")
wholesale=pd.read_excel('wholesale2024.xlsx')

"""
The function takes in label, date, and wholesale_hub.

label is the unique identifier for each plan (66b672c7f74f57eb270fd506)

Date can be specific (2024-01-01) or created by generate_date_with_options using a specific month and weekday/weekend

Wholesale region naming rules are:
{'Midwest':'Indiana Hub RT Peak',
'Northwest':'Mid C Peak',
'New England':'Nepool MH DA LMP Peak',
'Northern California':'NP15 EZ Gen DA LMP Peak',
'Southwest (Excluding Cali)':'Palo Verde Peak',
'PJM/Mid-Atlantic':'PJM WH Real Time Peak',
'Southern California':'SP15 EZ Gen DA LMP Peak'}
No Texas and Florida wholesale data available.
"""

"\nThe function takes in label, date, and wholesale_hub.\n\nlabel is the unique identifier for each plan (66b672c7f74f57eb270fd506)\n\nDate can be specific (2024-01-01) or created by generate_date_with_options using a specific month and weekday/weekend\n\nWholesale region naming rules are:\n{'Midwest':'Indiana Hub RT Peak',\n'Northwest':'Mid C Peak',\n'New England':'Nepool MH DA LMP Peak',\n'Northern California':'NP15 EZ Gen DA LMP Peak',\n'Southwest (Excluding Cali)':'Palo Verde Peak',\n'PJM/Mid-Atlantic':'PJM WH Real Time Peak',\n'Southern California':'SP15 EZ Gen DA LMP Peak'}\nNo Texas and Florida wholesale data available. \n"

In [5]:
def generate_date_with_options(month, weekday=True):
  import random
  from datetime import date, timedelta

  start = date(2024, month, 1)
  if month == 12:
    end = date(2024, 12, 31)
  else:
    end = date(2024, month + 1, 1) - timedelta(days=1)

  while True:
    delta_days = random.randint(0, (end - start).days)
    d = start + timedelta(days=delta_days)
    if weekday and d.weekday() < 5:
        return d
    if not weekday and d.weekday() >= 5:
        return d

def get_wholesale_prices(wholesale, hub, target_date):
    hub_dict={'Midwest':'Indiana Hub RT Peak',
    'Northwest':'Mid C Peak',
    'New England':'Nepool MH DA LMP Peak',
    'Northern California':'NP15 EZ Gen DA LMP Peak',
    'Southwest (Excluding Cali)':'Palo Verde Peak',
    'PJM/Mid-Atlantic':'PJM WH Real Time Peak',
    'Southern California':'SP15 EZ Gen DA LMP Peak'}
    target_date=pd.to_datetime(target_date)
    wholesale=wholesale[wholesale['Price hub']==hub_dict[hub]]
    wholesale.loc[:, 'Trade date'] = pd.to_datetime(wholesale['Trade date'])
    df = wholesale[wholesale['Trade date'] == target_date]

    if df.empty:
        # If no match, get next available date
        future_dates = wholesale[wholesale['Trade date'] > target_date]
        if future_dates.empty:
            return pd.DataFrame()  # no future data
        next_date = future_dates['Trade date'].min()
        df = wholesale[wholesale['Trade date'] == next_date]

    return df.iloc[0]


def get_plot_with_date(label: str, date:str, wholesale_hub:str):
  #############################################################################################
  #############################Retail Rates of the Day begins#############################
  #read data from db
  con = duckdb.connect("flattened.duckdb")
  df = con.execute("SELECT * FROM usurdb WHERE label = ?", (label,)).fetchdf()

  info=df[['label', 'eiaid', 'name', 'utility', 'description', 'servicetype','ratestructure']]
  periods=df[['label','energyweekdayschedule','energyweekendschedule']]

  #explode the rate structure
  months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
  periods = periods.explode(['energyweekdayschedule', 'energyweekendschedule'], ignore_index=True)
  periods["month"] = months * (len(periods)//12)

  hours=[str(_) for _ in list(range(24))]
  periods = periods.explode(['energyweekdayschedule', 'energyweekendschedule'], ignore_index=True)
  periods["hour"] = hours * (len(periods)//24)

  #transform to long format
  periods = periods.melt(
      id_vars=[c for c in periods.columns if c not in ["energyweekdayschedule","energyweekendschedule"]],
      value_vars=["energyweekdayschedule","energyweekendschedule"],
      var_name="day_type",
      value_name="period_id"
  )
  periods["day_type"] = periods["day_type"].map({
      "energyweekdayschedule": "weekday",
      "energyweekendschedule": "weekend"
  })

  #get rate info
  def get_hourly_values(label, date_str, periods, info):
    date = pd.to_datetime(date_str)
    month = date.strftime("%b")  # 'Jan', 'Feb', etc.
    day_type = "weekday" if date.weekday() < 5 else "weekend"

    # Select 24 rows from periods
    mask = (periods["label"] == label) & (periods["month"] == month) & (periods["day_type"] == day_type)
    period_rows = periods.loc[mask].sort_values("hour")  # Ensure 0-23 order

    # Get the rate structure for this label
    rate_str = info.loc[info["label"] == label, "ratestructure"].values[0]
    rate_dict = json.loads(rate_str)

    values = [
        rate_dict["energyrate"][f'period{row["period_id"]}']
        for _, row in period_rows.iterrows()
    ]
    return values
  rate_of_the_day=get_hourly_values(label, date, periods, info)
  tier_lists = {f'tier{i}': [] for i in range(10)}
  for hour_dict in rate_of_the_day:
      for i in range(10):
          tier_name = f'tier{i}'
          if tier_name in hour_dict:
              tier_lists[tier_name].append(hour_dict[tier_name])
          else:
              pass

  # Combine all tier lists into one list
  all_tiers = []
  for tier_name in sorted(tier_lists.keys()):
      if tier_lists[tier_name]:  # Only add if the tier has data
          all_tiers.append(tier_lists[tier_name])
  print(f"Total # of tiers collected: {len(all_tiers)}")

  all_tiers = [[x['rate']+x['adj'] for x in tier] for tier in all_tiers]

  ############################Retail Rates of the Day finished############################
  #############################################################################################

  #############################################################################################
  ###########################Wholesale Rates of the Day begins############################
  #read wholesale excel somewhere in advance
  wholesale_values=get_wholesale_prices(wholesale, wholesale_hub, date)
  max_wholesale=wholesale_values['High price $/MWh']/1000
  min_wholesale=wholesale_values['Low price $/MWh']/1000
  avg_wholesale=wholesale_values['Wtd avg price $/MWh']/1000
  ##########################Wholesale Rates of the Day finished###########################
  #############################################################################################

  #plotting begins
  data = []
  #from discrete, each-hour-as-a-point data, to continuous format
  for series_idx, tier in enumerate(all_tiers):
      for hour, value in enumerate(tier):
          data.append({
              'hour': hour,
              'value': value,
              'series': f'Tier {series_idx + 1}'
          })
          data.append({
              'hour': hour + 1,
              'value': value,
              'series': f'Tier {series_idx + 1}'
          })

  tmp = pd.DataFrame(data)

  # Create the main chart
  chart = alt.Chart(tmp).mark_line(strokeWidth=2).encode(
      x=alt.X('hour:Q',
              title='Hour of Day',
              scale=alt.Scale(domain=[0, 24]),
              axis=alt.Axis(tickCount=24, labelAngle=0)),
      y=alt.Y('value:Q', title='$ per unit'),
      color=alt.Color('series:N', title='Retail Price'),
      detail='series:N'
  ).properties(
      width=700,
      height=400,
      title='24-Hour Pricing Structure'
  )

  # Create reference lines for max and min wholesale
  reference_data = pd.DataFrame([
      {'hour': 0, 'value': max_wholesale, 'line': f'Max Wholesale ({max_wholesale})'},
      {'hour': 24, 'value': max_wholesale, 'line': f'Max Wholesale ({max_wholesale})'},
      {'hour': 0, 'value': min_wholesale, 'line': f'Min Wholesale ({min_wholesale})'},
      {'hour': 24, 'value': min_wholesale, 'line': f'Min Wholesale ({min_wholesale})'},
      {'hour': 0, 'value': avg_wholesale, 'line': f'Avg Wholesale ({avg_wholesale})'},
      {'hour': 24, 'value': avg_wholesale, 'line': f'Avg Wholesale ({avg_wholesale})'}
  ])

  reference_lines = alt.Chart(reference_data).mark_line(
      strokeWidth=1,
      opacity=0.5,
      strokeDash=[5, 5]
  ).encode(
      x='hour:Q',
      y='value:Q',
      color=alt.Color('line:N', title='Wholesale Prices')
  )


  # Combine the charts
  final_chart = reference_lines + chart

  return final_chart

2024-03-05
2024-07-21


In [8]:
get_plot_with_date('66b672c7f74f57eb270fd506',generate_date_with_options(month=5, weekday=True),wholesale_hub='Midwest')

CatalogException: Catalog Error: Table with name usurdb does not exist!
Did you mean "pragma_database_list"?

LINE 1: SELECT * FROM usurdb WHERE label = ?
                      ^